# Hyper parameters optimization #

## Parameters cell ##

Parameters are overiddent by papermill when run inside DVC stages



In [34]:
n_folds = 5 # this sets global setting of which how many bootstraps to use
n_hold_out = 1
repeats = 5
n_trials = 50
importance_type = "split"
trait = "lifespan"#"lifespan"

debug_local = True #to use local version

In [35]:
from pathlib import Path
import sys
import inspect

local = (Path("..") / "yspecies").resolve()
if debug_local and local.exists():
  sys.path.insert(0, Path("..").as_posix())
  #sys.path.insert(0, local.as_posix())
  print("extending pathes with local yspecies")
  print(sys.path)
  %load_ext autoreload
  %autoreload 2

extending pathes with local yspecies
['..', '..', '/data/sources/yspecies/notebooks', '/data/miniconda3/envs/yspecies/lib/python38.zip', '/data/miniconda3/envs/yspecies/lib/python3.8', '/data/miniconda3/envs/yspecies/lib/python3.8/lib-dynload', '', '/data/miniconda3/envs/yspecies/lib/python3.8/site-packages', '/data/miniconda3/envs/yspecies/lib/python3.8/site-packages/IPython/extensions', '/home/anton/.ipython']
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
from dataclasses import dataclass, replace
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
import seaborn as sns
import matplotlib.pyplot as plt

In [37]:
from typing import *
from yspecies.dataset import *
from yspecies.utils import *
from yspecies.workflow import TupleWith, Repeat, Collect
from yspecies.config import *
from yspecies.preprocess import FeatureSelection, DataExtractor
from yspecies.partition import DataPartitioner, PartitionParameters
from yspecies.selection import ShapSelector
from yspecies.tuning import Tune, MultiObjectiveResults
from yspecies.models import ResultsCV, CrossValidator
from yspecies.results import FeatureSummary
import optuna
from optuna import Study, Trial

In [38]:
#settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
import pprint
pp = pprint.PrettyPrinter(indent=4)

#charts settings
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
%matplotlib auto
plt.ioff()
set_matplotlib_formats('svg')

Using matplotlib backend: agg


### Loading data ###
Let's load data from species/genes/expressions selected by select_samples.py notebook

In [39]:
from pathlib import Path
locations: Locations = Locations("./") if Path("./data").exists() else Locations("../")

In [40]:
import logging
log = logging.getLogger('server_logger')
log.setLevel(logging.DEBUG)
fh = logging.FileHandler(locations.logs / "optimization.log")

In [41]:
from loguru import logger

logger.add(locations.logs / "optimization_errors.log", backtrace=True, diagnose=True)
logger.add(locations.logs / "optimization.log", rotation="12:00")     # New file is created each day at noon

4

In [42]:
selection = FeatureSelection(
    samples = ["tissue","species"], #samples metadata to include
    species =  [], #species metadata other then Y label to include
    exclude_from_training = ["species"],  #exclude some fields from LightGBM training
    to_predict = trait, #column to predict
    categorical = ["tissue"],
    select_by = "shap",
    importance_type =  importance_type,
    feature_perturbation = "tree_path_dependent"
)
selection

Samples metadata,Species metadata,Genes,Predict label,not_validated species
"['tissue', 'species']",[],all,lifespan,[]


In [43]:
data = ExpressionDataset.from_folder(locations.interim.dir / "stage_2" / "input_data")
data

expressions,genes,species,samples,Genes Metadata,Species Metadata
"(408, 90)","(90, 37)",38,408,"(90, 2)","(38, 18)"


## Setting up ShapSelector ##

Deciding on selection parameters (which fields to include, exclude, predict)

In [44]:
partition_params = PartitionParameters(n_folds, n_hold_out, 2,   42)

In [45]:
def load_study(trait: str):
    url = f'sqlite:///' +str((locations.interim.optimization / (trait+"_2.sqlite")).absolute())
    print('loading (if exists) study from '+url)
    storage = optuna.storages.RDBStorage(
        url=url
        #engine_kwargs={'check_same_thread': False}
    )
    return optuna.multi_objective.study.create_study(directions=['maximize','minimize','maximize'], storage = storage, study_name = f"{trait}_r2_huber_kendall", load_if_exists = True)

study = load_study(trait)
study
#metrics, params = MultiObjectiveResults.from_study(study).best_metrics_params_r2()
#metrics, params

loading (if exists) study from sqlite:////data/sources/yspecies/notebooks/../data/interim/optimization/lifespan_2.sqlite


create_study is experimental (supported from v1.4.0). The interface can change in the future.
NSGAIIMultiObjectiveSampler is experimental (supported from v1.5.0). The interface can change in the future.
RandomMultiObjectiveSampler is experimental (supported from v1.4.0). The interface can change in the future.
[I 2020-08-31 09:10:35,510] Using an existing study with name 'lifespan_r2_huber_kendall' instead of creating a new one.
MultiObjectiveStudy is experimental (supported from v1.4.0). The interface can change in the future.


In [46]:
def objective_parameters(trial: Trial) -> dict:
    return {
        'objective': 'regression',
        'metric': {'mae', 'mse', 'huber'},
        'verbosity': -1,
        'boosting_type': trial.suggest_categorical('boosting_type', ['dart', 'gbdt']),
        'lambda_l1': trial.suggest_uniform('lambda_l1', 0.01, 3.0),
        'lambda_l2': trial.suggest_uniform('lambda_l2', 0.01, 3.0),
        'max_leaves': trial.suggest_int("max_leaves", 15, 25),
        'max_depth': trial.suggest_int('max_depth', 3, 8),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.5, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.3, 1.0),
        'learning_rate': trial.suggest_uniform('learning_rate', 0.01, 0.1),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 3, 8),
        'drop_rate': trial.suggest_uniform('drop_rate', 0.1, 0.3),
        "verbose": -1
    }
optimization_parameters = objective_parameters

In [47]:
from yspecies.workflow import SplitReduce

def side(i: int):
    print(i)
    return i

prepare_partition = SplitReduce(
    outputs = DataPartitioner(), 
    split = lambda x: [(x[0], replace(partition_params, seed=side(x[2])))], 
    reduce = lambda x, output: (output[0], x[1]) 
)                               
partition_and_cv = Pipeline(
    [
        ("prepare partition", prepare_partition),
        ("shap_computation", ShapSelector()) #('crossvalidator', CrossValidator())        
    ]
)

partition_and_cv_repeat =  Pipeline([
    ("repeat_cv_pipe", Repeat(partition_and_cv, repeats, lambda x, i: [x[0], x[1], i] )),
    #("collect_mean", Collect(fold=lambda results: np.array([r.last("huber") for r in results]).mean()))
    ("collect_mean", Collect(fold=lambda results: (FeatureSummary(results).metrics_average.R2, FeatureSummary(results).metrics_average.huber, FeatureSummary(results).kendall_tau_abs_mean)))    
    ]
    )

p = Pipeline([
     ('extractor', DataExtractor()),
     ('tune', Tune(partition_and_cv_repeat, study = study, n_trials = n_trials, parameters_space = optimization_parameters))    
])

In [ ]:
results = p.fit_transform((data, selection))
results

MultiObjectiveStudy is experimental (supported from v1.4.0). The interface can change in the future.
FrozenMultiObjectiveTrial is experimental (supported from v1.4.0). The interface can change in the future.
MultiObjectiveStudy is experimental (supported from v1.4.0). The interface can change in the future.
FrozenMultiObjectiveTrial is experimental (supported from v1.4.0). The interface can change in the future.
FrozenMultiObjectiveTrial is experimental (supported from v1.4.0). The interface can change in the future.
MultiObjectiveTrial is experimental (supported from v1.4.0). The interface can change in the future.
MultiObjectiveStudy is experimental (supported from v1.4.0). The interface can change in the future.
FrozenMultiObjectiveTrial is experimental (supported from v1.4.0). The interface can change in the future.


0


2020-08-31 09:10:39.774 | INFO     | yspecies.selection:fit:78 - ===== fitting models with seed 0 =====
2020-08-31 09:10:39.776 | INFO     | yspecies.selection:fit:79 - PARAMETERS:
{'objective': 'regression', 'metric': {'mse', 'huber', 'mae'}, 'verbosity': -1, 'boosting_type': 'gbdt', 'lambda_l1': 0.8408976353362078, 'lambda_l2': 2.988126141377181, 'max_leaves': 19, 'max_depth': 3, 'feature_fraction': 0.584904668425096, 'bagging_fraction': 0.36258768829736376, 'learning_rate': 0.0213329832416484, 'min_data_in_leaf': 8, 'drop_rate': 0.12985939864611992, 'verbose': -1}
2020-08-31 09:10:39.782 | INFO     | yspecies.selection:fit:82 - SEED: 0 | FOLD: 0 | VALIDATION_SPECIES: ['Homo_sapiens', 'Equus_caballus']


Training until validation scores don't improve for 10 rounds
[250]	valid_0's l2: 2006.15	valid_0's huber: 19.4199	valid_0's l1: 21.9521
Did not meet early stopping. Best iteration is:
[250]	valid_0's l2: 2006.15	valid_0's huber: 19.4199	valid_0's l1: 21.9521


2020-08-31 09:10:40.010 | INFO     | yspecies.selection:fit:82 - SEED: 0 | FOLD: 1 | VALIDATION_SPECIES: ['Ursus_americanus', 'Phascolarctos_cinereus']


...

[250]	valid_0's l2: 49.6062	valid_0's huber: 3.01885	valid_0's l1: 3.70268


2020-08-31 09:14:53.981 | INFO     | yspecies.selection:fit:82 - SEED: 0 | FOLD: 2 | VALIDATION_SPECIES: ['Suricata_suricatta', 'Mus_spicilegus']


[250]	valid_0's l2: 16.5331	valid_0's huber: 1.69675	valid_0's l1: 2.20606


2020-08-31 09:14:54.515 | INFO     | yspecies.selection:fit:82 - SEED: 0 | FOLD: 3 | VALIDATION_SPECIES: ['Aotus_nancymaae', 'Macaca_fascicularis']


[250]	valid_0's l2: 26.6492	valid_0's huber: 2.04647	valid_0's l1: 2.59497


In [ ]:
results.best_trial_r2().values

In [ ]:
for i, t in enumerate(best):
    trait_path = locations.metrics.optimization / trait
    if not trait_path.exists():
        trait_path.mkdir()
    path = trait_path / f"{str(i)}.json"    
    print(f"writing parameters to {path}")
    with open(path, 'w') as f:
        params = t.params
        values = t.values
        to_write = {"number": t.number,"params": params, "metrics": {"R2":values[0], "huber": values[1], "kendall_tau": values[2]}}
        json.dump(to_write, f, sort_keys=True, indent=4)